In [1]:
import pandas as pd

In [2]:
train = pd.read_pickle("../data/processed/train_data.pkl")

In [3]:
train.columns.values

array(['building_id', 'timestamp', 'meter_reading', 'site_id',
       'square_feet', 'year_built', 'floor_count', 'air_temperature',
       'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr',
       'sea_level_pressure', 'wind_direction', 'wind_speed', 'meter_0',
       'meter_1', 'meter_2', 'meter_3', 'primary_use_Education',
       'primary_use_Entertainment/public assembly',
       'primary_use_Food sales and service', 'primary_use_Healthcare',
       'primary_use_Lodging/residential',
       'primary_use_Manufacturing/industrial', 'primary_use_Office',
       'primary_use_Other', 'primary_use_Parking',
       'primary_use_Public services', 'primary_use_Religious worship',
       'primary_use_Retail', 'primary_use_Services',
       'primary_use_Technology/science', 'primary_use_Utility',
       'primary_use_Warehouse/storage', 'timeofday_sin', 'timeofday_cos',
       'dayofweek_sin', 'dayofweek_cos', 'dayofyear_sin', 'dayofyear_cos',
       'wind_direction_sin', 'wind_directio

In [4]:
train.dtypes

building_id                                           int16
timestamp                                    datetime64[ns]
meter_reading                                       float32
site_id                                                int8
square_feet                                           int32
year_built                                          float16
floor_count                                         float16
air_temperature                                     float16
cloud_coverage                                      float16
dew_temperature                                     float16
precip_depth_1_hr                                   float16
sea_level_pressure                                  float16
wind_direction                                      float16
wind_speed                                          float16
meter_0                                               uint8
meter_1                                               uint8
meter_2                                 

In [5]:
train.drop(columns=["timestamp"])

,building_id,meter_reading,site_id,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,...,primary_use_Utility,primary_use_Warehouse/storage,timeofday_sin,timeofday_cos,dayofweek_sin,dayofweek_cos,dayofyear_sin,dayofyear_cos,wind_direction_sin,wind_direction_cos
0,0,0.000000,0,7432,2008.0,NaN,25.000000,6.0,20.000000,NaN,...,0,0,0.000000,1.000000,-0.433884,-0.900969,1.716633e-02,0.999853,0.000000,0.0
1,1,0.000000,0,2720,2004.0,NaN,25.000000,6.0,20.000000,NaN,...,0,0,0.000000,1.000000,-0.433884,-0.900969,1.716633e-02,0.999853,0.000000,0.0
2,2,0.000000,0,5376,1991.0,NaN,25.000000,6.0,20.000000,NaN,...,0,0,0.000000,1.000000,-0.433884,-0.900969,1.716633e-02,0.999853,0.000000,0.0
3,3,0.000000,0,23685,2002.0,NaN,25.000000,6.0,20.000000,NaN,...,0,0,0.000000,1.000000,-0.433884,-0.900969,1.716633e-02,0.999853,0.000000,0.0
4,4,0.000000,0,116607,1975.0,NaN,25.000000,6.0,20.000000,NaN,...,0,0,0.000000,1.000000,-0.433884,-0.900969,1.716633e-02,0.999853,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20125600,1444,8.750000,15,19619,1914.0,NaN,1.700195,NaN,-5.601562,-1.0,...,0,0,-0.258819,0.965926,-0.974928,-0.222521,-2.449294e-16,1.000000,-0.000074,-1.0
20125601,1445,4.825000,15,4298,NaN,NaN,1.700195,NaN,-5.601562,-1.0,...,0,0,-0.258819,0.965926,-0.974928,-0.222521,-2.449294e-16,1.000000,-0.000074,-1.0
20125602,1446,0.000000,15,11265,1997.0,NaN,1.700195,NaN,-5.601562,-1.0,...,0,0,-0.258819,0.965926,-0.974928,-0.222521,-2.449294e-16,1.000000,-0.000074,-1.0
20125603,1447,159.574997,15,29775,2001.0,NaN,1.700195,NaN,-5.601562,-1.0,...,0,0,-0.258819,0.965926,-0.974928,-0.222521,-2.449294e-16,1.000000,-0.000074,-1.0
